In [1]:
import numpy as np
import gpkit
import astropy.constants as const
from gpkit import Model, Variable
from gpkit.nomials.math import SingleSignomialEquality,SignomialInequality
from gpkit.constraints.bounded import Bounded
gpkit.SIGNOMIALS_ENABLED = True
from collections import OrderedDict

In [52]:
muval = 3.98e14
alpha_1 = Variable("\\alpha_1", 0.6624)
alpha_2 = Variable("\\alpha_2", 0.06991)
alpha_3 = Variable("\\alpha_3", 1266.75, "s")
alpha_4 = Variable("\\alpha_4", 804.90, "s")

R_earth = Variable("R_{earth}", const.R_earth.value, "m", "earth radius")
eta_s = Variable("\\eta_s", 0.29, "-", "solar panel efficiency")
Q_s = Variable("Q_s", 1367, "W/m^2", "solar flux constant")
EbNo_req = Variable("EbNo_{req}", 40, "-", "ebno required")

G_Rx = Variable("G_{Rx}", "-", "ground receive gain")
D_Rx = Variable("D_{Rx}", 5.3, "m", "ground receive diamater")

D_Tx = Variable("D_{Tx}", "m", "spacecraft antenna diameter")
L_other = Variable("L_{other}", 9.772372209558107, "-", "line loss") #1.58489
k = Variable("k", const.k_B.value, "J/K", "Boltzman constant R/Na")
T_sys = Variable("T_{sys}", 135, "K", "system temperature")
N = Variable("N", 2000, '-', 'number of pixels along width')
B_ppx = Variable("B_{ppx}", 8, "bit", "bit per pixel")
eta = Variable("\\eta", 0.55, "-", "dish efficiency")
lambda_obs = Variable("\\lambda_{obs}", 500, "nm", "image payload wavelength")
e = Variable("e", 25, 'W/kg', 'power density of solar arrays') 
A_rho = Variable("A_{rho}", 10, "kg/m^2", "solar panel area density")

f = Variable("f", 2.2, "GHz", "transmit frequency")
lambda_com = Variable("\\lambda_{com}", "m", "transmit wavelength") #fake variable    
c = Variable("c", const.c.value, "m/s", "Speed of light in vacuum")
D_rho = Variable("D_{rho}", 100, "kg/m^1.5")
DTx_rho = Variable("DTx_{rho}", 2, "kg/m^1.5")
h_min = Variable('h_{min}', 400, 'km', 'minimum orbit altitude')

prop_rho = Variable('prop_{rho}', 500, 'kg*km', 'propellant altitude constant')
a_const = Variable('a_const', 200, 'km', 'altitude constant')
alpha_5 = Variable("\\alpha_5", 0.058823529411764705, "year/km")
alpha_6 = Variable("\\alpha_6", 1., "year/kg")
T = Variable("T", 3, "year", "lifetime")
P_load = Variable("P_{load}", 5, "W", "constant power drain")
m_extra = Variable("m_{extra}", 0.2, "kg", "constant mass")

mu = Variable("mu", muval, "m^3/s^2", "standard gravitational parameter")

####
Xp_req = Variable('Xp_{req}', 20, 'm', 'required resolution')
m_total = Variable("m_{total}", 'kg', 'spacecraft total mass')
h = Variable("h", "km", "orbit altitude")
d_f = Variable("d_f", "-", "not in-eclipse orbit fraction")
P_suppl = Variable("P_{suppl}", "W", "spacecraft total power")
P_Tx = Variable("P_{Tx}", "W", "transmit power")
r = Variable("r", "km", "max distance to ground station")
GS_f = Variable("GS_f", "-", "downlink orbit fraction")
P = Variable("P", "min", "period")
D = Variable("D", "m", "aperture")
A = Variable("A", "m^2", "surface area solar panels")
m_p = Variable("m_p", "kg", "propellant mass")


dummy = Variable("dummy", "km", "formulation variable for altitude")
dummy_bound = Variable("dummy_{bound}", 10000, "km")

constraints = [
    lambda_com == c/f,
    G_Rx == eta*(np.pi*D_Rx/lambda_com)**2,
    P_suppl == d_f*A*eta_s*Q_s,
    P_Tx + P_load <= P_suppl,
    EbNo_req == P_Tx*G_Rx*Xp_req*GS_f*P*eta*D_Tx**2/
    (L_other*k*T_sys*R_earth*2*np.pi*N*B_ppx*(4*r**2)),
    Xp_req == h*lambda_obs/D,
    d_f <= GS_f + 0.5,
    dummy >= R_earth+h,
    dummy <= dummy_bound,
    #GS_f <= alpha_1*h/R_earth + alpha_2,
    GS_f**0.133355 <= 0.295066*(h/R_earth)**0.0619879+0.294791*(h/R_earth)**0.061991+
    0.294739 * (h/R_earth)**0.0619928,
    #P == 2*np.pi*a**1.5/mu**0.5,
    P <= 2*np.pi*(alpha_3*h/R_earth + alpha_4),
    r**2 >= h**2 + 2*R_earth*h,
    D_rho*D**1.5 + DTx_rho*D_Tx**1.5 + A_rho*A + prop_rho/h + m_extra<= 0.8*m_total
    #h >= h_min,
    #T >= alpha_6*m_p + alpha_5*a
]
m = Model(m_total, constraints)
print(m.localsolve().table())

Beginning signomial solve.
Solving took 3 GP solves and 0.221 seconds.

Cost
----
 2.111 [kg] 

Free Variables
--------------
            A : 0.03646  [m**2] surface area solar panels
            D : 0.02281  [m]    aperture
       D_{Tx} : 0.2378   [m]    spacecraft antenna diameter
         GS_f : 0.1615          downlink orbit fraction
       G_{Rx} : 8211            ground receive gain
            P : 103.3    [min]  period
       P_{Tx} : 4.561    [W]    transmit power
    P_{suppl} : 9.561    [W]    spacecraft total power
\lambda_{com} : 0.1363   [m]    transmit wavelength
          d_f : 0.6615          not in-eclipse orbit fraction
        dummy : 1e+04    [km]   formulation variable for altitude
            h : 912.4    [km]   orbit altitude
    m_{total} : 2.111    [kg]   spacecraft total mass
            r : 3531     [km]   max distance to ground station

Constants
---------
      A_{rho} : 10         [kg/m**2]   solar panel area density
      B_{ppx} : 8          [bit]     